# Chapter 3 - python support

We will need some help from python to do our favour.

In [1]:
import getpass
import subprocess
import os
import shlex

In [2]:
# Declare variables here
CURRENT_DIR = os.getcwd()
ETC_PROFILE = "tmp_profile" # This is a file
SKEL_DIR = "tmp_skel" # This is a directory

In [3]:
# Type your username's password here.
user_password = getpass.getpass()

········


In [4]:
# Let's copy the /etc/profile and /etc/skel to your notebook's directory
cp_profile_cmd = f"echo {shlex.quote(user_password)} | \
                sudo -S cp /etc/profile {CURRENT_DIR}/{ETC_PROFILE} && \
                sudo -S cp -r /etc/skel {CURRENT_DIR}/{SKEL_DIR}"
cp_profile = subprocess.run(cp_profile_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

if cp_profile.returncode != 0:
    print(f"Failed to copy /etc/profile and /etc/skel to {CURRENT_DIR}")
    print("==========================")
    print(cp_profile.stdout.decode('utf-8'))
else:
    print("Operation Complete! Successfully copying all these files and directory.")
    
    # Now, takes the ownership comrade!!
    current_uid = os.geteuid()
    take_owner_cmd = f"echo {shlex.quote(user_password)} | \
                    sudo -S chown -R {current_uid}:{current_uid} {shlex.quote(ETC_PROFILE)} {shlex.quote(SKEL_DIR)}"
    take_owner = subprocess.run(take_owner_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    if take_owner.returncode != 0:
        print(f"Failed to take ownership for /etc/profile and /etc/skel from root")
        print("==========================")
        print(take_owner.stdout.decode('utf-8'))
    else:
        print("Operation Complete! Successfully taking ownership from root.")

print("Now you can step back to your old notebook.")
    

Operation Complete! Successfully copying all these files and directory.
Operation Complete! Successfully taking ownership from root.
Now you can step back to your old notebook.


You can remove the folder and files using below command.

In [5]:
# Recursively remove dir
def _rmdir(path):
    try:   
        for root, dirs, files in os.walk(path, topdown=False):
            for file_name in files:
                os.remove(os.path.join(root, file_name))
            for dir_name in dirs:
                os.rmdir(os.path.join(root, dir_name))
        
        # Now, remove me.
        os.rmdir(path)
        return 0
    except Exception as e:
        return -1
    
def _rmfile(path):
    try:
        os.remove(path)
        return 0
    except FileNotFoundError as fe:
        return -1
    except Exception as e:
        return -99

In [6]:
rm_skell_dir = _rmdir(f"./{SKEL_DIR}")
rm_profile = _rmfile(f"./{ETC_PROFILE}")

print("Successfully removing /etc/skel dir" if rm_skell_dir == 0 \
      else "Oh no... Failed removing /etc/skel dir.")
print("Successfully removing /etc/profile" if rm_profile == 0 \
      else "Oh no.. File was gone..." if rm_profile == 0 \
      else "Oh no.. ??!??!")

Successfully removing /etc/skel dir
Successfully removing /etc/profile
